### Final EWMACD Method

In [1]:
# globals (dev)
FOLDER_MODULES = r'C:\Users\Lewis\Documents\GitHub\tenement-tools\modules'  
FOLDER_SHARED = r'C:\Users\Lewis\Documents\GitHub\tenement-tools\shared'
GRP_LYR_FILE = r"C:\Users\Lewis\Documents\GitHub\tenement-tools\arc\lyr\group_template.lyrx"

# set gdal global environ
import os
os.environ['GDAL_DISABLE_READDIR_ON_OPEN'] = 'EMPTY_DIR'
os.environ['CPL_VSIL_CURL_ALLOWED_EXTENSIONS '] = 'tif'
os.environ['VSI_CACHE '] = 'TRUE'
os.environ['GDAL_HTTP_MULTIRANGE '] = 'YES'
os.environ['GDAL_HTTP_MERGE_CONSECUTIVE_RANGES '] = 'YES'

# also set rasterio env variables
rasterio_env = {
    'GDAL_DISABLE_READDIR_ON_OPEN': 'EMPTY_DIR',
    'CPL_VSIL_CURL_ALLOWED_EXTENSIONS':'tif',
    'VSI_CACHE': True,
    'GDAL_HTTP_MULTIRANGE': 'YES',
    'GDAL_HTTP_MERGE_CONSECUTIVE_RANGES': 'YES'
}

# safe imports
import sys                      # arcgis comes with these
import shutil                   # arcgis comes with these
import datetime                 # arcgis comes with these
import numpy as np              # arcgis comes with these
import pandas as pd             # arcgis comes with these
import arcpy                    # arcgis comes with these
import tempfile                 # arcgis comes with these
#import matplotlib.pyplot as plt
import smtplib
import mimetypes
from datetime import datetime   # arcgis comes with these
from email.message import EmailMessage
from email.utils import make_msgid



# risky imports (not native to arcgis)
try:
    import xarray as xr
    import dask
    import rasterio
    import pystac_client
    import osr
    import json
    from scipy.signal import savgol_filter
    from odc import stac
    from osgeo import gdal
    from osgeo import ogr
    from osgeo import osr
except:
    arcpy.AddError('Python libraries xarray, dask, rasterio, pystac, or odc not installed.')
    raise # return

# import tools
try:
    # shared folder
    sys.path.append(FOLDER_SHARED)
    import arc, satfetcher, tools

    # module folder
    sys.path.append(FOLDER_MODULES)
    import nrt, cog_odc, cog
except:
    arcpy.AddError('Could not find tenement tools python scripts (modules, shared).')
    raise
    
# disable future warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
warnings.simplefilter(action='ignore', category=dask.array.core.PerformanceWarning)
    
# grab parameter values 
in_feat = r"C:\Users\Lewis\Desktop\nrt_projects\test2\monitoring_areas.gdb\monitoring_areas"
in_ongoing = False
in_time_interval = 24
#in_email_from = parameters[3].value                # email from 
#in_smtp_server = parameters[4].value               # email smtp server 
#in_smtp_port = parameters[5].value                 # email smtp port 
#in_smtp_username = parameters[6].value             # email smtp username 
#in_smtp_password = parameters[7].value             # email smtp password 


# # # # #
# notify user and set up progress bar
#arcpy.AddMessage('Beginning NRT Monitoring of areas.')
#arcpy.SetProgressor(type='step', 
                    #message='Preparing parameters...',
                    #min_range=0, max_range=20)
        
# set up initial continous monitoring var
continue_monitoring = True

# check if time interval is > 0
#in_time_interval = in_time_interval * 60 * 60
#if in_time_interval <= 0:
    #arcpy.AddError('Time interval must be above 0 hours.')
    #raise


# # # # #
# notify and increment progress bar
#arcpy.SetProgressorLabel('Preparing parameters...')
#arcpy.SetProgressorPosition(1)

# get path to monitoring areas feature
feat_desc = arcpy.Describe(in_feat)
in_feat = os.path.join(feat_desc.path, feat_desc.name)


# # # # #
# notify and increment progress bar
#arcpy.SetProgressorLabel('Validating monitoring areas...')
#arcpy.SetProgressorPosition(2)

# validate monitoring area feature class
try:
    nrt.validate_monitoring_areas(in_feat)
except:
    arcpy.AddError('Monitoring areas feature is invalid.')
    raise # return

    
# # # # #
# notify and increment progress bar
#arcpy.SetProgressorLabel('Loading monitoring area json data...')
#arcpy.SetProgressorPosition(2)
    
# prepare path to expected json file
in_path = os.path.dirname(in_feat)
in_path = os.path.splitext(in_path)[0]
in_path = os.path.dirname(in_path)
#in_data_path = os.path.join(in_path, 'data.json')
    
    
# # # # #
# notify and increment progress bar
#arcpy.SetProgressorLabel('Loading monitoring area features...')
#arcpy.SetProgressorPosition(2)

# set required fields
fields = [
    'area_id', 
    'platform', 
    's_year', 
    'e_year', 
    'index', 
    'persistence', 
    'rule_1_min_conseqs', 
    'rule_1_inc_plateaus', 
    'rule_2_min_zone', 
    'rule_3_num_zones', 
    'ruleset', 
    'alert', 
    'method',
    'alert_direction', 
    'email', 
    'ignore', 
    'color_border',
    'color_fill',
    'global_id', 
    'SHAPE@'
]

# get feature count and data
try:
    #feats = arcpy.da.SearchCursor(in_feat, fields)
    feats = []
    with arcpy.da.SearchCursor(in_feat, fields) as cursor:
        for row in cursor:
            feats.append(row)
except:
    arcpy.AddError('Could not open monitoring areas feature.')
    raise # return

OSError: "C:\Users\Lewis\Desktop\nrt_projects\test2\monitoring_areas.gdb\monitoring_areas" does not exist

In [2]:
from importlib import reload
reload(nrt)

<module 'nrt' from 'C:\\Users\\Lewis\\Documents\\GitHub\\tenement-tools\\modules\\nrt.py'>

In [84]:
# todo:
# remove field rule 1 inc plateaus ... ?
# remove field rule 2 bidirection. we do this via alert dir now - DONE
# change rule 2 min stdv to min zone - DONE
# consider a 'negative remover in positive areas' and vice versa. consider during alert?... add as extra alert dirs? - DONE
# append all mon area field info to netcdf attr, check at start of run for change, delete cube if change
# check if ruleset contains rule without a value entered during area creation (fields accepts nulls) - FORCED REQUIRED!

In [85]:
# 'inc_any':      'Incline only (any)', 
# 'dec_any':      'Decline only (any)', 
# 'inc_pos':      'Incline only (+ zones only)', 
# 'dec_neg':      'Decline only (- zones only)', 
# 'both_any':     'Incline or Decline (any)',
# 'both_pos_neg': 'Incline or Decline (+/- zones only)',

In [86]:
# tips
# use higher persistence for dynamic (1) and lower for static (0.5)
# use decline in - or pos in + only for dynamic to avoid new regime shifts triggering alarm
# turn off spikes for dynamic

In [ ]:
def execute(self, parameters, messages):
    """
    Executes the NRT Monitor Areas module.
    """

    # safe imports
    import os, sys
    import time
    import datetime   
    import numpy as np
    import tempfile
    import arcpy

    # set rasterio env 
    rio_env = {
        'GDAL_DISABLE_READDIR_ON_OPEN': 'EMPTY_DIR',
        'CPL_VSIL_CURL_ALLOWED_EXTENSIONS': 'tif',
        'VSI_CACHE': 'TRUE',
        'GDAL_HTTP_MULTIRANGE': 'YES',
        'GDAL_HTTP_MERGE_CONSECUTIVE_RANGES': 'YES'
        }

    # apply rio env settings 
    for k, v in rio_env.items():
        os.environ[k] = v

    # risky imports (not native to arcgis)
    try:
        import xarray as xr
        import dask
        import rasterio
        import pystac_client
        from odc import stac
    except Exception as e:
        arcpy.AddError('Python libraries xarray, dask, rasterio, odc not installed.')
        arcpy.AddMessage(str(e))
        return

    # import tools
    try:
        # shared folder
        sys.path.append(FOLDER_SHARED)
        import arc, satfetcher, tools

        # module folder
        sys.path.append(FOLDER_MODULES)
        import cog_odc, nrt
    except Exception as e:
        arcpy.AddError('Could not find tenement tools python scripts (modules, shared).')
        arcpy.AddMessage(str(e))
        return

    # disable future warnings
    import warnings
    warnings.simplefilter(action='ignore', category=FutureWarning)
    warnings.simplefilter(action='ignore', category=RuntimeWarning)
    warnings.simplefilter(action='ignore', category=UserWarning)
    warnings.simplefilter(action='ignore', category=dask.array.core.PerformanceWarning)   

    # grab parameter values 
    in_feat = parameters[0]                            # input monitoring area feature
    in_continuous = parameters[1].value                # continuous monitoring
    in_num_days = parameters[2].value                  # days between checks
    in_send_email = parameters[3].value                # send email alerts
    in_email_host = parameters[4].value                # host email address
    in_email_server = parameters[5].value              # host email server
    in_email_port = parameters[6].value                # host email port
    in_email_username = parameters[7].value            # host email username
    in_email_password = parameters[8].value            # host email password



    # # # # #
    # notify and set on-going progess bar
    arcpy.SetProgressor('default', 'Validating monitoring area geodatabase...')

    # get full path to existing monitoring areas
    feat_desc = arcpy.Describe(parameters[0].value)
    in_feat = os.path.join(feat_desc.path, feat_desc.name)

    try:
        # check if input is valid (error if invalid)
        nrt.validate_monitoring_areas(in_feat)
    except Exception as e:
        arcpy.AddError('Monitoring areas feature is incompatible, see messages for details.')
        arcpy.AddMessage(str(e))
        return



    # # # # #
    # notify and set on-going progess bar
    arcpy.SetProgressor('default', 'Obtaining and validating monitoring areas...')

    # set required fields
    fields = [
        'area_id', 
        'platform', 
        's_year', 
        'e_year', 
        'index', 
        'persistence', 
        'rule_1_min_conseqs', 
        'rule_1_inc_plateaus', 
        'rule_2_min_zone', 
        'rule_3_num_zones', 
        'ruleset', 
        'alert', 
        'method',
        'alert_direction', 
        'email', 
        'ignore', 
        'color_border', 
        'color_fill', 
        'global_id', 
        'SHAPE@'
    ]

    try:
        # get features
        with arcpy.da.SearchCursor(in_feat, fields) as cursor:
            feats = [row for row in cursor]
    except Exception as e:
        arcpy.AddError('Could not read monitoring areas.')
        arcpy.AddMessage(str(e))
        return

    # check if feature exists 
    if len(feats) == 0:
        arcpy.AddError('No monitoring areas exist.')
        return



    # # # # #
    # notify and set on-going progess bar
    arcpy.SetProgressor('default', 'Converting features to monitoring areas objects...')

    # prepare path to current geodatabase folder
    in_path = os.path.dirname(in_feat)
    in_path = os.path.splitext(in_path)[0]
    in_path = os.path.dirname(in_path)

    try:
        # iter feature and convert to monitoring area objects
        areas = [nrt.MonitoringArea(feat, path=in_path) for feat in feats]
    except Exception as e:
        arcpy.AddError('Could not convert to monitoring area objects.')
        arcpy.AddMessage(str(e))
        return

    # check if areas exist
    if len(areas) == 0:
        arcpy.AddError('No monitoring area objects exist.')
        return



    # # # # #
    # notify and set on-going progess bar
    arcpy.AddMessage('Beginning monitoring cycle...')
    arcpy.SetProgressor('default', 'Monitoring existing areas...')

    # check continuous and num days are valid 
    if in_continuous not in [True, False]:
        arcpy.AddError('Did not provide continuous monitoring option.')
        return
    elif in_num_days < 1:
        arcpy.AddError('Number of days between cycles must be >= 1.')
        return

    # prepare monitor iterator 
    iterations = 99999 if in_continuous else 1

    # begin monitoring cycles...
    for _ in range(iterations):

        # iter monitoring area objects...
        for area in areas:

            # # # # #
            # notify user and set up progress bar
            arcpy.AddMessage(u'\u200B')
            arcpy.AddMessage('Starting area {}.'.format(area.area_id))
            arcpy.SetProgressor(type='step',
                                message='Starting area: {}...'.format(area.area_id),
                                min_range=0, max_range=30)

            # notify and skip area if user requested
            if area.ignore is True:
                arcpy.AddMessage('Area set to ignore, skipping.')
                continue



            # # # # #
            # notify user and increment progress bar
            arcpy.AddMessage('Validating monitoring area...')
            arcpy.SetProgressorLabel('Validating monitoring area...') 
            arcpy.SetProgressorPosition(1) 

            try:
                # validate area, skip if error
                area.validate_area()
            except Exception as e:
                arcpy.AddWarning('Area is invalid, see messages for details.')
                arcpy.AddMessage(str(e))
                continue



            # # # # #
            # notify user and increment progress bar
            arcpy.AddMessage('Checking for existing area NetCDF...')
            arcpy.SetProgressorLabel('Checking for existing area NetCDF...') 
            arcpy.SetProgressorPosition(2)

            try:
                # get old xr, if none set old to none, if error set old to none
                area.set_old_xr()

                # notify if no old xr and proceed
                if area.ds_old is None:
                    arcpy.AddMessage('No existing NetCDF found.')
                    pass
                else:
                    arcpy.AddMessage('Found existing NetCDF.')
                    pass
            except:
                arcpy.AddMessage('No existing area NetCDF, fetching a new NetCDF.')
                pass



            # # # # #
            # notify user and increment progress bar
            arcpy.AddMessage('Validating existing NetCDF...')
            arcpy.SetProgressorLabel('Validating existing NetCDF...') 
            arcpy.SetProgressorPosition(3)

            try:
                # validate old xr, skip if none or attrs differ, skip if error
                area.validate_old_xr()

                # notify if no old xr and proceed
                if area.ds_old is None:
                    arcpy.AddMessage('Existing NetCDF is invalid (or does not exist).')
                    pass
                else:
                    arcpy.AddMessage('Existing NetCDF is valid.')
                    pass
            except:
                arcpy.AddWarning('Attributes of existing NetCDF have changed, resetting.')
                pass



            # # # # #
            # notify user and increment progress bar
            arcpy.AddMessage('Obtaining new satellite data, if exists...')
            arcpy.SetProgressorLabel('Obtaining new satellite data, if exists...') 
            arcpy.SetProgressorPosition(4)

            try:
                # get all sat data, set new xr to new dates if old exists else all new, on error set new to none and skip
                area.set_new_xr()  

                # skip if no new dates, else notify and proceed
                if area.new_xr_dates_found() is False:
                    arcpy.AddMessage('No new satellite images, skipping area.')
                    continue
                else:
                    arcpy.AddMessage('Found {} new satellite images.'.format(len(area.ds_new['time'])))
                    pass
            except Exception as e:
                arcpy.AddWarning('Could not obtain new satellite data, see messages.')
                arcpy.AddMessage(str(e))
                continue    



            # # # # #
            # notify user and increment progress bar
            arcpy.AddMessage('Removing satellite images with clouds...')
            arcpy.SetProgressor('default', 'Removing satellite images with clouds...')

            try:
                # apply fmask on new xr. if error or no dates, skip
                area.apply_new_xr_fmask()

                # check if any valid images remain, else notify and skip
                if area.new_xr_dates_found() is False:
                    arcpy.AddMessage('No cloud-free satellite images remain, skipping area.')
                    continue 
                else:
                    arcpy.AddMessage('Found {} cloud-free satellite images'.format(len(area.ds_new['time'])))
                    pass
            except Exception as e:
                arcpy.AddWarning('Could not apply fmask, see messages.')
                arcpy.AddMessage(str(e))
                continue   



            # # # # #
            # notify user and increment progress bar
            arcpy.AddMessage('Calculating vegetation index for new satellite images...')
            arcpy.SetProgressorLabel('Calculating vegetation index for new satellite images...') 
            arcpy.SetProgressorPosition(6)

            try:
                # calculate vege index for new xr. if error or no dates, skip
                area.apply_new_xr_index() 
            except Exception as e:
                arcpy.AddWarning('Could not calculate vegetation index, see messages.')
                arcpy.AddMessage(str(e))
                continue



            # # # # #
            # notify user and on-going progress bar
            arcpy.AddMessage('Downloading new satellite data, please wait...')
            arcpy.SetProgressor('default', 'Downloading new satellite data, please wait...')

            try:
                # load new xr, skip if error
                area.load_new_xr() 
            except Exception as e:
                arcpy.AddWarning('Could not download satellite data, see messages.')
                arcpy.AddMessage(str(e))
                continue



            # # # # #
            # notify user and increment progress bar
            arcpy.AddMessage('Removing edge pixels via mask...')
            arcpy.SetProgressorLabel('Removing edge pixels via mask...') 
            arcpy.SetProgressorPosition(8)

            try:
                # remove edge pixels. if error or empty, return original xr
                area.remove_new_xr_edges() 
            except Exception as e:
                arcpy.AddWarning('Could not remove edge pixels, proceeding without mask.')
                arcpy.AddMessage(str(e))
                pass



            # # # # #
            # notify user and increment progress bar
            arcpy.AddMessage('Reducing new satellite images down to temporal medians...')
            arcpy.SetProgressorLabel('Reducing new satellite images down to temporal medians...') 
            arcpy.SetProgressorPosition(9)

            try:
                # reduce new xr to median value per date. skip if error
                area.reduce_new_xr() 
            except Exception as e:
                arcpy.AddWarning('Could not reduce satellite images, see messages.')
                arcpy.AddMessage(str(e))
                continue



            # # # # #
            # notify user and increment progress bar
            arcpy.AddMessage('Interpolating missing pixels...')
            arcpy.SetProgressorLabel('Interpolating missing pixels...') 
            arcpy.SetProgressorPosition(10)

            try:
                # interp nans. if any remain, drop. if empty, error and skip
                area.interp_new_xr_nans() 
            except Exception as e:
                arcpy.AddWarning('Could not interpolate missing pixels, see messages.')
                arcpy.AddMessage(str(e))
                continue



            # # # # #
            # notify user and increment progress bar
            arcpy.AddMessage('Appending required variables...')
            arcpy.SetProgressorLabel('Appending required variables...') 
            arcpy.SetProgressorPosition(11)

            try:
                # append required vars to new xr. skip if error
                area.append_new_xr_vars() 
            except Exception as e:
                arcpy.AddWarning('Could not append variables, see messages.')
                arcpy.AddMessage(str(e))
                continue



            # # # # #
            # notify user and increment progress bar
            arcpy.AddMessage('Combining old and new satellite images, if required...')
            arcpy.SetProgressorLabel('Combining old and new satellite images, if required...') 
            arcpy.SetProgressorPosition(12)

            try:
                # combine old and new xrs. if no old xr, new xr used. if error, skip
                area.set_cmb_xr() 
            except Exception as e:
                arcpy.AddWarning('Could not combine old and new images, see messages.')
                arcpy.AddMessage(str(e))
                continue



            # # # # #
            # notify user and increment progress bar
            arcpy.AddMessage('Removing vegetation spikes in combined images...')
            arcpy.SetProgressorLabel('Removing vegetation spikes in combined images...') 
            arcpy.SetProgressorPosition(13)

            try:
                # replace cmb spikes, interp removed. set result to var veg_clean. skip if error or empty.
                area.fix_cmb_xr_spikes() 
            except Exception as e:
                arcpy.AddWarning('Could not remove spikes, see messages.')
                arcpy.AddMessage(str(e))
                continue



            # # # # #
            # notify user and increment progress bar
            arcpy.AddMessage('Smoothing vegetation in combined images...')
            arcpy.SetProgressorLabel('Smoothing vegetation spike in combined images...') 
            arcpy.SetProgressorPosition(14)

            try:
                # smooth cmb veg_clean var with savitsky filter. skip if error.
                area.smooth_cmb_xr_index() 
            except Exception as e:
                arcpy.AddWarning('Could not smooth vegetation, see messages.')
                arcpy.AddMessage(str(e))
                continue



            # # # # #
            # notify user and increment progress bar
            arcpy.AddMessage('Building analysis dataset...')
            arcpy.SetProgressorLabel('Building analysis dataset...') 
            arcpy.SetProgressorPosition(15)

            try:
                # build anl xr, remove pre-training years. skip if error.
                area.set_anl_xr()
            except Exception as e:
                arcpy.AddWarning('Could not build analysis dataset, see messages.')
                arcpy.AddMessage(str(e))
                continue



            # # # # #
            # notify user and increment progress bar
            arcpy.AddMessage('Performing change detection on analysis dataset...')
            arcpy.SetProgressorLabel('Performing change detection on analysis dataset...') 
            arcpy.SetProgressorPosition(16)

            try:
                # detect static, dynamic change via anl xr, skip if error.
                area.detect_change_anl_xr()
            except Exception as e:
                arcpy.AddWarning('Could not perform change detection, see messages.')
                arcpy.AddMessage(str(e))
                continue



            # # # # #
            # notify user and increment progress bar
            arcpy.AddMessage('Smoothing change detection on analysis dataset...')
            arcpy.SetProgressorLabel('Smoothing change detection on analysis dataset...') 
            arcpy.SetProgressorPosition(17)

            try:
                # smooth static, dynamic change via anl xr, skip if error.
                area.smooth_anl_xr_change()
            except Exception as e:
                arcpy.AddWarning('Could not smooth change detection data, see messages.')
                arcpy.AddMessage(str(e))
                continue



            # # # # #
            # notify user and increment progress bar
            arcpy.AddMessage('Transfering old data to analysis dataset...')
            arcpy.SetProgressorLabel('Transfering old data to analysis dataset...') 
            arcpy.SetProgressorPosition(18)

            try:
                # transfer old xr change values to anl xr, skip if error.
                area.transfer_old_to_anl_xr()
            except Exception as e:
                arcpy.AddWarning('Could not transfer data, see messages.')
                arcpy.AddMessage(str(e))
                continue



            # # # # #
            # notify user and increment progress bar
            arcpy.AddMessage('Calculating zones...')
            arcpy.SetProgressorLabel('Calculating zones...') 
            arcpy.SetProgressorPosition(19)

            try:
                # build zones, skip if error.
                area.build_zones()
            except Exception as e:
                arcpy.AddWarning('Could not calculate zones, see messages.')
                arcpy.AddMessage(str(e))
                continue



            # # # # #
            # notify user and increment progress bar
            arcpy.AddMessage('Calculating rule one, two and three...')
            arcpy.SetProgressorLabel('Calculating rule one, two and three...') 
            arcpy.SetProgressorPosition(20)

            try:
                # build rule one, two, three. skip if error.
                area.build_rule_one()
                area.build_rule_two()
                area.build_rule_three()
            except Exception as e:
                arcpy.AddWarning('Could not calculate rules, see messages.')
                arcpy.AddMessage(str(e))
                continue



            # # # # #
            # notify user and increment progress bar
            arcpy.AddMessage('Calculating alerts...')
            arcpy.SetProgressorLabel('Calculating alerts...') 
            arcpy.SetProgressorPosition(21)

            try:
                # build alerts, skip if error.
                area.build_alerts()
            except Exception as e:
                arcpy.AddWarning('Could not calculate alerts, see messages.')
                arcpy.AddMessage(str(e))
                continue



            # # # # #
            # notify user and increment progress bar
            arcpy.AddMessage('Transfering analysis data to combined dataset...')
            arcpy.SetProgressorLabel('Transfering analysis data to combined dataset...') 
            arcpy.SetProgressorPosition(22)

            try:
                # transfer anl xr to cmb xr. skip if error.
                area.transfer_anl_to_cmb_xr()
            except Exception as e:
                arcpy.AddWarning('Could not transfer analysis data, see messages.')
                arcpy.AddMessage(str(e))
                continue



            # # # # #
            # notify user and increment progress bar
            arcpy.AddMessage('Appending area attributes on to combined dataset...')
            arcpy.SetProgressorLabel('Appending area attributes on to combined dataset...') 
            arcpy.SetProgressorPosition(23)

            try:
                # append area field attrs on to cmb xr. skip if error.
                area.append_cmb_xr_attrs()
            except Exception as e:
                arcpy.AddWarning('Could not append area attributes, see messages.')
                arcpy.AddMessage(str(e))
                continue   



            # # # # #
            # notify user and increment progress bar
            arcpy.AddMessage('Generating area info, html and graph...')
            arcpy.SetProgressorLabel('Generating area info, html and graph...') 
            arcpy.SetProgressorPosition(24)

            try:
                # generate alert info, html, graph data. skip if error.
                area.set_alert_data()
            except Exception as e:
                arcpy.AddWarning('Could not generate alert information, html, graphs.')
                arcpy.AddMessage(str(e))
                continue  



            # # # # #
            # notify user and increment progress bar
            arcpy.AddMessage('Refreshing monitoring area symbology...')
            arcpy.SetProgressorLabel('Refreshing monitoring area symbology...') 
            arcpy.SetProgressorPosition(25)

            try:
                # refresh monitoring area symbology. skip if error.
                area.refresh_area_symbology()
            except Exception as e:
                arcpy.AddWarning('Could not refresh symbology, see messages.')
                arcpy.AddMessage(str(e))
                pass  



            # # # # #
            # notify user and increment progress bar
            arcpy.AddMessage('Exporting combined NetCDF...')
            arcpy.SetProgressorLabel('Exporting combined data...') 
            arcpy.SetProgressorPosition(26)

            try:
                area.export_cmb_xr()
            except Exception as e:
                arcpy.AddWarning('Could not export NetCDF, see messages.')
                arcpy.AddMessage(str(e))
                continue  



            # # # # #
            # notify user and show on-going progress bar               
            arcpy.AddMessage('Cooling off for 10 seconds...')
            arcpy.SetProgressor('default', 'Cooling off for 10 seconds...')

            # pause for 10 seconds 
            for sec in range(10):
                time.sleep(1.0)



        # # # # #
        # notify and set on-going progess bar
        arcpy.AddMessage('Preparing and sending email alert, if requested...')
        arcpy.SetProgressor('default', 'Preparing and sending email alert, if requestted...')

        # if requested...
        if in_send_email is True:            
            try:
                # prepare email alerts for each area and send, skip if error
                nrt.email_alerts(areas, 
                                 host_email=in_email_host, 
                                 host_server=in_email_server, 
                                 host_port=in_email_port, 
                                 host_user=in_email_username, 
                                 host_pass=in_email_password)
            except Exception as e:
                arcpy.AddWarning('Could not send email, see messages for details.')
                arcpy.AddMessage(str(e))
                pass



        # # # # #
        # notify and set on-going progess bar
        arcpy.AddMessage(u'\u200B')
        arcpy.AddMessage('Resetting monitoring areas...')
        arcpy.SetProgressor('default', 'Resetting monitoring areas...')

        try:
            # reset each monitor area's internal data 
            for area in areas:
                area.reset()
        except Exception as e:
            arcpy.AddWarning('Could not reset areas, see messages for details.')
            arcpy.AddMessage(str(e))
            pass



        # # # # #
        # notify and set on-going progess bar
        arcpy.AddMessage('Waiting for next cycle, if requested...')
        arcpy.SetProgressor('default', 'Waiting for next cycle, if requested...')

        # if requested, were done!
        if not in_continuous:
            break 

        # prepare new progressor for wait
        arcpy.SetProgressor(type='step', 
                            message='Waiting for next monitoring cycle...', 
                            min_range=0, 
                            max_range=in_num_days * 86400)

        # begin iter until next cycle
        for sec in range(in_num_days * 86400):
            arcpy.SetProgressorPosition(sec)
            time.sleep(1.0)

            # ensure we leave on tool cancel properly
            if arcpy.env.isCancelled:
                sys.exit(0)



    # # # # #
    # notify and increment progress bar
    arcpy.SetProgressorLabel('Finalising process...')
    arcpy.SetProgressorPosition(3)

    # notify user
    arcpy.AddMessage('Monitored areas successfully.')
    return

In [1]:
import xarray as xr

In [4]:
ds = xr.open_dataset(r"C:\Users\Lewis\Desktop\test\a3.nc")
ds = ds.load()

In [11]:
ds['change_latest_time_decline'].mean()

<xarray.DataArray 'change_latest_time_decline' ()>
array(-0.00067677)
Coordinates:
    spatial_ref  int32 3577